In [1]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
from datetime import datetime

In [2]:
import config
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

https://stackoverflow.com/questions/40023931/how-to-get-real-estate-data-with-idealista-api

In [3]:
def get_oauth_token(): 
    url = "https://api.idealista.com/oauth/token"    
    apikey= API_KEY #sent by idealist
    secret= API_SECRET #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]
    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [4]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [5]:
# values: es, it, pt
country = 'es' 
# search language for summary. values: es, it, pt, en, ca
locale = 'es' 
language = 'es' 
# items per page: 50 as maximum allowed
max_items = '50'
# values: sale, rent
operation = 'sale'
# values: homes, offices, premises, garages, bedrooms
property_type = 'homes'
# allowed values by property type:
# garages: distance, price, street, photos, publicationDate, modificationDate (rent only), weigh, pricedown
# premises: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2 (rent only), weigh, pricedown
# offices: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2, weigh, pricedown
# homes: distance, price, street, photos, publicationDate, modificationDate, size, floor, rooms, ratioeurm2 (sólo alquiler), weigh, pricedown
# rooms: distance, price, street, photos, publicationDate, modificationDate, floor
order = 'priceDown' 
# geographic coordinates (WGS84) (latitude,longitude)
center = '43.3773,-8.39852' 
# distance to center, in metres (ratio)
distance = '5000'
# values: asc, desc
sort = 'desc'
# 
maxPrice = '300000'
# numPage = 1

In [6]:
df = pd.DataFrame()
for i in range(1,15):
    numPage = str(i)
    url = ('https://api.idealista.com/3.5/'+country+
            '/search?operation='+operation+
            '&locale='+locale+
            '&maxItems='+max_items+
            '&order='+order+
            '&center='+center+
            '&distance='+distance+
            '&maxPrice='+maxPrice+
            '&propertyType='+property_type+
            '&sort='+sort+ 
            '&numPage='+numPage+
            '&language='+language)
    result = search_api(get_oauth_token(), url)
    df_temp = pd.json_normalize(result, record_path = ["elementList"])
    df = pd.concat([df, df_temp])
now = datetime.now()
date_time = now.strftime("%m%d%Y_%H:%M")
df.to_csv('idealista_search' + str(date_time) + '.csv')

In [7]:
print(len(df))
df

700


,propertyCode,thumbnail,numPhotos,floor,price,propertyType,operation,size,exterior,rooms,...,suggestedTexts.subtitle,suggestedTexts.title,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,detailedType.subTypology,externalReference,neighborhood,parkingSpace.parkingSpacePrice,labels,newDevelopmentFinished
0,95334761,https://img3.idealista.com/blur/WEB_LISTING/0/...,18,2,190000.0,flat,sale,93.0,True,4,...,"Los Castros - Castrillón, A Coruña",Piso en Oza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,99041337,https://img3.idealista.com/blur/WEB_LISTING/0/...,39,4,140000.0,duplex,sale,120.0,True,5,...,"Someso - Matogrande, A Coruña",Dúplex en Calle Petunias s/n,True,True,duplex,NaN,NaN,NaN,NaN,NaN
2,99130874,https://img3.idealista.com/blur/WEB_LISTING/0/...,1,5,65000.0,flat,sale,60.0,True,3,...,"Os Mallos, A Coruña",Piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99142116,https://img3.idealista.com/blur/WEB_LISTING/0/...,19,3,140000.0,duplex,sale,111.0,True,5,...,"Someso - Matogrande, A Coruña",Dúplex en Calle das Petunias,NaN,NaN,duplex,5435A,NaN,NaN,NaN,NaN
4,99023805,https://img3.idealista.com/blur/WEB_LISTING/90...,40,4,205000.0,flat,sale,115.0,True,4,...,"Falperra-Santa Lucía, A Coruña",Piso en Falperra,True,True,NaN,NaN,Falperra-Santa Lucía,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,81752587,https://img3.idealista.com/blur/WEB_LISTING/0/...,27,NaN,210000.0,chalet,sale,124.0,False,4,...,"Sagrada Familia, A Coruña",Casa independiente en Calle Río Limia,NaN,NaN,independantHouse,20081B,NaN,NaN,NaN,NaN
46,99462628,https://img3.idealista.com/blur/WEB_LISTING/0/...,45,1,170000.0,flat,sale,90.0,False,3,...,"Falperra-Santa Lucía, A Coruña","Piso en Sinforiano López, 54",NaN,NaN,NaN,049.36070,Falperra-Santa Lucía,NaN,NaN,NaN
47,98371003,https://img3.idealista.com/blur/WEB_LISTING/0/...,28,9,200000.0,flat,sale,90.0,True,2,...,"Riazor - Los Rosales, A Coruña",Piso en Calle Colombia,NaN,NaN,NaN,3421-07871,NaN,NaN,NaN,NaN
48,98710367,https://img3.idealista.com/blur/WEB_LISTING/0/...,38,6,295000.0,flat,sale,109.0,True,3,...,"Falperra-Santa Lucía, A Coruña",Piso en Calle Costa da Unión,NaN,NaN,NaN,5402RED,Falperra-Santa Lucía,NaN,NaN,NaN
